In [44]:
!pip install wandb
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.layers import Dense,MaxPool2D,Flatten,Conv2D
import os
from tensorflow.keras import layers
from wandb.keras import WandbCallback
import wandb

In [45]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
dataset_path="/content/drive/MyDrive/Persian_Recognition"
width=height=224
batch_size=64


Data_Augmention

In [47]:
data_generator=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2
)
train_data=data_generator.flow_from_directory(
   dataset_path ,
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training'

)

val_data=data_generator.flow_from_directory(
    dataset_path,
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='validation'
)


Found 2338 images belonging to 2 classes.
Found 584 images belonging to 2 classes.


In [48]:
base_model=tf.keras.applications.ResNet50(
    input_shape=(width,height,3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [49]:
for layer in base_model.layers:
  layer.trainable=False


In [50]:
model=tf.keras.Sequential([base_model,layers.Dense(2,activation='softmax')])


In [51]:
wandb.init(project="Persian_recognition")

In [52]:
config = wandb.config
config.learning_rate = 0.001

In [53]:
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    metrics=['accuracy']
)

In [54]:
model.fit(train_data,steps_per_epoch=train_data.samples/batch_size,
          validation_data=val_data,
          validation_steps=val_data.samples/batch_size,
          epochs=10,callbacks = [WandbCallback()])



Epoch 1/10
36/36 [==============================] - 734s 20s/step - loss: 0.5923 - accuracy: 0.7160 - val_loss: 0.5283 - val_accuracy: 0.7397


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/10
36/36 [==============================] - 29s 796ms/step - loss: 0.5104 - accuracy: 0.7553 - val_loss: 0.4848 - val_accuracy: 0.7723
Epoch 3/10
36/36 [==============================] - 30s 807ms/step - loss: 0.4938 - accuracy: 0.7772 - val_loss: 0.4932 - val_accuracy: 0.7432
Epoch 4/10
36/36 [==============================] - 29s 801ms/step - loss: 0.4371 - accuracy: 0.8062 - val_loss: 0.4376 - val_accuracy: 0.8048
Epoch 5/10
36/36 [==============================] - 29s 802ms/step - loss: 0.4214 - accuracy: 0.8114 - val_loss: 0.4405 - val_accuracy: 0.8322
Epoch 6/10
36/36 [==============================] - 29s 803ms/step - loss: 0.4069 - accuracy: 0.8212 - val_loss: 0.4164 - val_accuracy: 0.7962
Epoch 7/10
36/36 [==============================] - 30s 821ms/step - loss: 0.3950 - accuracy: 0.8366 - val_loss: 0.4778 - val_accuracy: 0.7637
Epoch 8/10
36/36 [==============================] - 30s 806ms/step - loss: 0.3747 - accuracy: 0.8400 - val_loss: 0.3847 - val_accuracy: 0.8253

In [55]:
model.save('/content/model.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
